In [7]:
import numpy as np
import pandas as pd
import get_data

In [82]:
X, y = get_data.get_all_data()

X shape:  (70691, 2)
y shape:  (70691,)


In [83]:
test_strings = ['http', 'Trump', 'BREAKING', 'scandal', ]

def flag_vocab(X, y, substrings):
    # search of specific phrases - not currently used
    substrings = substrings
    bool_matrix = np.zeros((len(y), len(substrings)), dtype=int)

    def search_substrings(row, index):
        for j, substring in enumerate(substrings):
            # print(row.name)
            for column_name in X.columns:
                if substring in row[column_name]:
                # if substring in row['title'] or substring in row['text']:
                    bool_matrix[index, j] = 1

    # Apply the function to each row of the DataFrame
    X.apply(lambda row: search_substrings(row, X.index.get_loc(row.name)), axis=1)

    def count_words(text):
        return len(text.split())

    # Apply the function to count words in each string
    X['word count text'] = X['text'].apply(lambda x: count_words(x))
    X['word count title'] = X['title'].apply(lambda x: count_words(x))

    result_df = pd.DataFrame(bool_matrix, columns=substrings, index=X.index)
    merged_df = X.join(result_df)

    return merged_df

X_new = flag_vocab(X, y, test_strings)
print('avg title length: real: ', X_new.loc[y == 1, 'word count title'].mean(), 'fake: ', X_new.loc[y == 0, 'word count title'].mean())
print('avg text length: real: ', X_new.loc[y == 1, 'word count text'].mean(), 'fake: ', X_new.loc[y == 0, 'word count text'].mean())
for string in test_strings:
    print(string, ': real: ', (X_new[string] & y).sum(), ', fake: ', (X_new[string] & ~y).sum())


avg title length: fake:  11.046756733598013 real:  13.427886771300448
avg text length: fake:  575.7649881465825 real:  522.6766816143498
http : real:  4451 , fake:  302
Trump : real:  17120 , fake:  15542
BREAKING : real:  1278 , fake:  20


In [72]:
X_new

,title,text,word count text,word count title,http,Trump,BREAKING
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,871,18,0,0,0
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last n...",34,18,0,0,0
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,1291,16,0,0,0
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",329,16,0,0,0
5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,244,13,0,0,0
...,...,...,...,...,...,...,...
72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,735,11,0,1,0
72130,WATCH: Giuliani Demands That Democrats Apologi...,"You know, because in fantasyland Republicans n...",604,10,0,1,0
72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,466,10,0,0,0
72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,559,10,0,1,0


In [68]:
real_trump = (X_new['Trump'] & y).sum()
fake_trump = (X_new['Trump'] & ~y).sum()
real_breaking = (X_new['BREAKING'] & y).sum()
fake_breaking = (X_new['BREAKING'] & ~y).sum()

print(real_trump, fake_trump)
print(real_breaking, fake_breaking)

17120 15542
1278 20


In [85]:
X_new.isnull().sum().sum()

0